# Topic Modeling
### Without Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 19:27:16.122812: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 19:27:16.122833: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [2]:
n_components = 15 #Topics To generate
n = 40 #Transcripts to use
max_df=20
min_df=3
n_terms = 10

## Load Data

In [3]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [6]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                if y['NAMES'][0] == 'MATT':
                    text = ' '.join(y['UTTERANCES'])

                    text = clean_text(text)
                    list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [7]:
df = create_df_multi(n)
print(df.shape)
df.head()

(29284, 1)


,0
0,to tonight episode of bunch of ass voice actor...
1,like jump into tonight episode have announceme...
2,appearance now on
3,get to laughter
4,mention before thank again friend at for spons...


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [8]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray().transpose(),
                   index=tfidf.get_feature_names_out())
tfidf_df.tail()

(29284, 3229)


,0,1,2,3,4,5,6,7,8,9,...,29274,29275,29276,29277,29278,29279,29280,29281,29282,29283
zealotry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zenith,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zigzag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zip,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray().transpose(),
                   index=cv.get_feature_names_out())
cv_df.tail()

(29284, 3229)


,0,1,2,3,4,5,6,7,8,9,...,29274,29275,29276,29277,29278,29279,29280,29281,29282,29283
zealotry,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zenith,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zigzag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [10]:
def print_top_terms(n_components, topics, terms, n_terms = 10):
    for x in range(n_components):
        topic = x
        components = topics[:,topic]
        top_term_indices = components.argsort()[-n_terms:]
        top_terms = np.array(terms)[top_term_indices]
        
        print(f'Topic {x}:\t{top_terms.tolist()}')


## PCA

In [11]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['paralyze', 'rapidly', 'tight', 'teach', 'angel', 'dot', 'fox', 'veil', 'roil', 'mute']
Topic 1:	['teach', 'angel', 'dot', 'varn', 'k', 'colony', 'fox', 'roil', 'veil', 'mute']
Topic 2:	['mausoleum', 'undercroft', 'ignore', 'syla', 'bloodline', 'reading', 'charm', 'brink', 'wealth', 'cellar']
Topic 3:	['batter', 'dispel', 'proceed', 'timing', 'waterfall', 'transform', 'wealth', 'rider', 'hip', 'cellar']
Topic 4:	['compare', 'intel', 'farmland', 'fulfill', 'horseback', 'hatch', 'ambush', 'phrase', 'protector', 'slaver']
Topic 5:	['fortress', 'slavery', 'halfle', 'pilgrimage', 'overview', 'mutate', 'paladin', 'abomination', 'enemy', 'quest']
Topic 6:	['canine', 'bony', 'starch', 'ankle', 'pupil', 'accent', 'puff', 'switch', 'linger', 'portrait']
Topic 7:	['seize', 'faith', 'blacksmithy', 'downpour', 'selling', 'sprinkle', 'anvil', 'banging', 'stride', 'ware']
Topic 8:	['robot', 'goatee', 'necromancer', 'demeanor', 'mourn', 'agreement', 'argument', 'bazaar', 'daughter', 'resurre

In [12]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['marching', 'treant', 'robot', 'film', 'tour', 'completion', 'birthday', 'lend', 'pardon', 'buddy']
Topic 1:	['tour', 'tinker', 'jostle', 'relieve', 'marching', 'clinking', 'honestly', 'buddy', 'timing', 'sleight']
Topic 2:	['deflect', 'crossroad', 'selling', 'downpour', 'blacksmithy', 'banging', 'anvil', 'sprinkle', 'stride', 'ware']
Topic 3:	['auto', 'drool', 'credit', 'imp', 'thread', 'lend', 'completion', 'minus', 'paralyze', 'automatically']
Topic 4:	['stealthing', 'unleash', 'scimitar', 'ready', 'glaive', 'minus', 'loop', 'automatically', 'treant', 'jester']
Topic 5:	['merch', 'bush', 'flutter', 'jester', 'unmove', 'jet', 'feat', 'general', 'froth', 'drool']
Topic 6:	['flutter', 'greatsword', 'goliath', 'shriek', 'yank', 'marching', 'deflect', 'froth', 'drool', 'treant']
Topic 7:	['lend', 'froth', 'scan', 'greatsword', 'jester', 'unlock', 'drool', 'eel', 'theory', 'melee']
Topic 8:	['spatter', 'budge', 'froth', 'melee', 'drool', 'shock', 'treant', 'credit', 'eel', 'minu

## LDA

In [13]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['reform', 'shade', 'hate', 'vein', 'cat', 'paralyze', 'mutter', 'buddy', 'rapidly', 'manacle']
Topic 1:	['latch', 'unmove', 'troupe', 'marble', 'caravan', 'plenty', 'loosely', 'credit', 'monk', 'regard']
Topic 2:	['theory', 'basis', 'training', 'refuse', 'sulfur', 'shriek', 'glove', 'illusion', 'muck', 'egg']
Topic 3:	['transpire', 'scan', 'season', 'sizzle', 'suck', 'brass', 'eel', 'podcast', 'erinye', 'flare']
Topic 4:	['escort', 'cap', 'affix', 'singe', 'mustache', 'anchor', 'tiger', 'enemy', 'crumble', 'wealth']
Topic 5:	['melee', 'flick', 'splatter', 'merch', 'shrink', 'patrol', 'hunch', 'spike', 'simultaneously', 'jewelry']
Topic 6:	['chew', 'pearl', 'perch', 'clock', 'recovery', 'completion', 'gnarl', 'jet', 'contest', 'mechanism']
Topic 7:	['brighten', 'regardless', 'warn', 'glimmer', 'red', 'gauntlet', 'curtain', 'pressure', 'donation', 'goliath']
Topic 8:	['instinctively', 'tab', 'dip', 'dome', 'yank', 'lend', 'accomplish', 'female', 'tar', 'subscriber']
Topic 9:	['

In [14]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['filter', 'current', 'standpoint', 'sentinel', 'subscriber', 'groove', 'properly', 'completion', 'plenty', 'regardless']
Topic 1:	['fellow', 'attune', 'rate', 'flavor', 'backhand', 'hiss', 'procure', 'expression', 'manacle', 'mainly']
Topic 2:	['situate', 'possibility', 'coal', 'benefit', 'casting', 'jet', 'camera', 'react', 'general', 'muck']
Topic 3:	['audio', 'lower', 'apology', 'tune', 'transpire', 'longbow', 'goliath', 'honestly', 'theory', 'jester']
Topic 4:	['d', 'pardon', 'tiger', 'scenario', 'wonder', 'daylight', 'fine', 'erinye', 'melee', 'treant']
Topic 5:	['melt', 'hat', 'medicine', 'disengage', 'distraction', 'bush', 'shriek', 'anchor', 'tar', 'enchant']
Topic 6:	['loosely', 'shrink', 'freak', 'suit', 'slap', 'limb', 'equipment', 'simultaneously', 'egg', 'unlock']
Topic 7:	['protrude', 'palm', 'blink', 'suck', 'basis', 'deckhand', 'trick', 'wand', 'tinker', 'credit']
Topic 8:	['lend', 'flare', 'paralyze', 'physicality', 'singing', 'greatsword', 'feat', 'scan', 'i

## SVD

In [15]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['mood', 'enslave', 'statement', 'bulk', 'rob', 'paltry', 'brand', 'varn', 'k', 'colony']
Topic 1:	['paralyze', 'rapidly', 'tight', 'teach', 'angel', 'dot', 'fox', 'veil', 'roil', 'mute']
Topic 2:	['taunt', 'dispel', 'batter', 'proceed', 'transform', 'timing', 'waterfall', 'rider', 'hip', 'cellar']
Topic 3:	['mausoleum', 'undercroft', 'ignore', 'syla', 'bloodline', 'reading', 'charm', 'brink', 'wealth', 'cellar']
Topic 4:	['compare', 'intel', 'farmland', 'fulfill', 'horseback', 'hatch', 'ambush', 'phrase', 'protector', 'slaver']
Topic 5:	['fortress', 'slavery', 'halfle', 'pilgrimage', 'overview', 'mutate', 'paladin', 'abomination', 'enemy', 'quest']
Topic 6:	['permeate', 'starch', 'bony', 'ankle', 'pupil', 'accent', 'puff', 'switch', 'linger', 'portrait']
Topic 7:	['robot', 'necromancer', 'goatee', 'demeanor', 'mourn', 'agreement', 'bazaar', 'argument', 'daughter', 'resurrection']
Topic 8:	['orchard', 'faith', 'blacksmithy', 'downpour', 'selling', 'sprinkle', 'anvil', 'banging

In [16]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['film', 'deflect', 'eel', 'jester', 'birthday', 'muck', 'discount', 'pardon', 'spatter', 'buddy']
Topic 1:	['eel', 'spatter', 'timing', 'automatically', 'lumber', 'fucking', 'discount', 'marching', 'jester', 'sleight']
Topic 2:	['merch', 'sleight', 'selling', 'blacksmithy', 'downpour', 'banging', 'anvil', 'sprinkle', 'stride', 'ware']
Topic 3:	['auto', 'theory', 'greatsword', 'thread', 'enchant', 'tour', 'merch', 'scan', 'paralyze', 'automatically']
Topic 4:	['tour', 'flutter', 'yank', 'paralyze', 'egg', 'cock', 'spatter', 'minus', 'general', 'jester']
Topic 5:	['manacle', 'feat', 'imp', 'sizzle', 'eel', 'jester', 'discount', 'enchant', 'froth', 'drool']
Topic 6:	['shriek', 'flutter', 'unlock', 'jester', 'cock', 'fine', 'froth', 'general', 'drool', 'treant']
Topic 7:	['completion', 'lend', 'individually', 'print', 'equipment', 'fine', 'discount', 'tour', 'melee', 'theory']
Topic 8:	['lend', 'warn', 'greatsword', 'cock', 'unleash', 'shock', 'jester', 'discount', 'melee', 'eel'

## NMF

In [17]:
nmf = NMF(n_components=n_components, max_iter=500)
topics = nmf.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Topic 0:	['infiltrate', 'mood', 'statement', 'bulk', 'rob', 'paltry', 'brand', 'k', 'varn', 'colony']
Topic 1:	['paralyze', 'rapidly', 'tight', 'teach', 'angel', 'dot', 'fox', 'veil', 'roil', 'mute']
Topic 2:	['clip', 'atop', 'dispel', 'batter', 'proceed', 'timing', 'transform', 'waterfall', 'rider', 'hip']
Topic 3:	['undercroft', 'ignore', 'syla', 'myth', 'bloodline', 'reading', 'charm', 'brink', 'wealth', 'cellar']
Topic 4:	['compare', 'intel', 'farmland', 'fulfill', 'horseback', 'hatch', 'ambush', 'phrase', 'protector', 'slaver']
Topic 5:	['colony', 'slavery', 'halfle', 'pilgrimage', 'overview', 'mutate', 'paladin', 'abomination', 'enemy', 'quest']
Topic 6:	['permeate', 'bony', 'starch', 'ankle', 'pupil', 'accent', 'puff', 'switch', 'linger', 'portrait']
Topic 7:	['robot', 'necromancer', 'goatee', 'demeanor', 'mourn', 'agreement', 'argument', 'bazaar', 'daughter', 'resurrection']
Topic 8:	['seize', 'crossroad', 'downpour', 'selling', 'blacksmithy', 'sprinkle', 'anvil', 'banging', 's

In [18]:
nmf = NMF(n_components=n_components, max_iter=400)
topics = nmf.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 400 reached. Increase it to improve convergence.
  warnings.warn(


Topic 0:	['talent', 'web', 'inspire', 'intro', 'lend', 'robot', 'film', 'birthday', 'pardon', 'buddy']
Topic 1:	['spike', 'rider', 'hip', 'narrative', 'bruise', 'jostle', 'relieve', 'clinking', 'timing', 'sleight']
Topic 2:	['duration', 'crossroad', 'selling', 'blacksmithy', 'downpour', 'banging', 'anvil', 'sprinkle', 'stride', 'ware']
Topic 3:	['crackle', 'cancel', 'mute', 'nudge', 'confuse', 'fang', 'auto', 'thread', 'paralyze', 'automatically']
Topic 4:	['trick', 'wax', 'blip', 'example', 'copy', 'stealthing', 'scimitar', 'ready', 'loop', 'jester']
Topic 5:	['reload', 'composure', 'regain', 'rifle', 'spitting', 'waste', 'glisten', 'jet', 'froth', 'drool']
Topic 6:	['creaking', 'headache', 'gout', 'smolder', 'gravity', 'knot', 'reference', 'harry', 'shriek', 'treant']
Topic 7:	['contest', 'surrounding', 'fortune', 'pound', 'nostril', 'flavor', 'awkwardly', 'd', 'accomplish', 'credit']
Topic 8:	['increase', 'mm', 'slice', 'mentally', 'whatsoever', 'bearing', 'superiority', 'longsword'